# CIFAR100 Based Image classification Model Retrain

###  Train EfficientNet-B0 on CIFAR-100 using PyTorch

In [24]:
# Import required:
import os
import torch
import platform
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.models import efficientnet_b0

In [32]:
# Define hyperparameters:
epochs = 20
LEARNING_RATE = 0.001

In [34]:
# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Display GPU details if available
if device.type == "cuda":
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"CUDA Version: {torch.version.cuda}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    
# Display CPU details
print(f"CPU: {platform.processor()}")
print(f"System: {platform.system()} {platform.release()}")

Using device: cuda
GPU Name: Orin
CUDA Version: 12.6
GPU Memory: 7.99 GB
CPU: aarch64
System: Linux 5.15.148-tegra


In [35]:
# Define transforms
transform = transforms.Compose([
    transforms.Resize((32, 32)), 
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [36]:
# Load CIFAR-100 dataset
trainset = torchvision.datasets.CIFAR100(root="./data", train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testset = torchvision.datasets.CIFAR100(root="./data", train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [37]:
# Load pre-trained EfficientNet-B0
model = efficientnet_b0(weights="IMAGENET1K_V1")  
model.classifier[1] = nn.Linear(1280, 100)  # Modify last layer for CIFAR-100 classes
model.to(device)

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [38]:
# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [39]:
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Compute accuracy
        _, predicted = torch.max(outputs, 1)  # Get class with highest score
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / len(trainloader)
    epoch_accuracy = 100 * correct / total

    print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")


Epoch 1/20, Loss: 2.8823, Accuracy: 28.67%
Epoch 2/20, Loss: 2.0048, Accuracy: 45.74%
Epoch 3/20, Loss: 1.7164, Accuracy: 52.59%
Epoch 4/20, Loss: 1.5243, Accuracy: 56.76%
Epoch 5/20, Loss: 1.3745, Accuracy: 60.59%
Epoch 6/20, Loss: 1.2429, Accuracy: 63.82%
Epoch 7/20, Loss: 1.1407, Accuracy: 66.44%
Epoch 8/20, Loss: 1.0486, Accuracy: 68.89%
Epoch 9/20, Loss: 0.9453, Accuracy: 71.27%
Epoch 10/20, Loss: 0.8968, Accuracy: 72.77%
Epoch 11/20, Loss: 0.8232, Accuracy: 74.67%
Epoch 12/20, Loss: 0.7765, Accuracy: 76.02%
Epoch 13/20, Loss: 0.7189, Accuracy: 77.62%
Epoch 14/20, Loss: 0.6651, Accuracy: 79.40%
Epoch 15/20, Loss: 0.6656, Accuracy: 79.54%
Epoch 16/20, Loss: 0.6802, Accuracy: 79.13%
Epoch 17/20, Loss: 0.6061, Accuracy: 81.05%
Epoch 18/20, Loss: 0.5460, Accuracy: 82.80%
Epoch 19/20, Loss: 0.5251, Accuracy: 83.31%
Epoch 20/20, Loss: 0.4863, Accuracy: 84.54%


In [40]:
# Define base model path
base_path = "/home/aman-nvidia/My_files/cv_projects/image_classification_webGUI/efficientnet_cifar100"
file_extension = ".pth"
model_path = base_path + file_extension

# Function to find the next available filename
def get_available_filename(base_path, file_extension):
    counter = 1
    new_path = f"{base_path}{file_extension}"
    
    while os.path.exists(new_path):  # Check if file exists
        new_path = f"{base_path}_{counter}{file_extension}"  # Append counter
        counter += 1
    
    return new_path

# Get a unique filename
model_path = get_available_filename(base_path, file_extension)

# Save the model
torch.save(model.state_dict(), model_path)
print(f"Model saved successfully as: {model_path}")

Model saved successfully as: /home/aman-nvidia/My_files/cv_projects/image_classification_webGUI/efficientnet_cifar100_2.pth
